In [38]:
words = open('interstellar.txt','r').read()

In [60]:
words = words.lower()

In [61]:
words[:100]

"ï»¿1\nwell, my dad was a farmer.\num, like everybody else back then.\nof course, he didn't start that w"

In [63]:
words.find("don't let me leave")

63262

In [64]:
words[63262:63262+30]

"don't let me leave, murph!\ndon"

In [65]:
import matplotlib.pyplot as plt
chars = sorted(list(set(''.join(words))))

In [66]:
print(chars)
len(chars)

['\n', ' ', '!', '"', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '©', '»', '¿', 'ã', 'ï']


51

In [67]:
vocab_size = len(chars)


In [254]:
#encoding
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}


In [255]:
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: ''.join([itos[s] for s in l])

decode(encode("hi murph"))

'hi murph'

In [256]:
import torch
data = torch.tensor(encode(words),dtype=torch.long)


In [257]:
print(data.shape)

torch.Size([69581])


In [258]:
n = int(0.9*(len(words)))
train_data = data[:n]
val_data = data[n:]

In [277]:
block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y
xb, yb = get_batch('train')
print(xb.shape)
print(xb)

print(yb.shape)
print(yb)
    

torch.Size([4, 8])
tensor([[35, 24, 22, 39, 38,  0, 20, 37],
        [34, 34, 35,  5,  0, 44, 34, 40],
        [25, 39,  5,  1, 39, 34,  1, 27],
        [24,  1, 42, 20, 44,  7,  0, 35]])
torch.Size([4, 8])
tensor([[24, 22, 39, 38,  0, 20, 37, 24],
        [34, 35,  5,  0, 44, 34, 40,  1],
        [39,  5,  1, 39, 34,  1, 27, 24],
        [ 1, 42, 20, 44,  7,  0, 35, 31]])


In [278]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLLM(torch.nn.Module):
    def __init__(self,embd_dims):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,embd_dims)
        #self.layer = nn.Linear(embd_dims,vocab_size)
    def forward(self,x,targets=None): #x = [B,T],targets = [B,T] too
        embd = self.embedding(x)#[B,T,C]
        #out = self.layer(embd) #[B,T,T]
        if targets is None:
            loss = None
        else:
            B,T,C = embd.shape
            embd = embd.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(embd,targets)
        return embd, loss    
    
    def generate(self,idx,max_new_tokens):
        for _ in range(max_new_tokens): ##[B,T]
            logits, loss = self(idx)
            logits = logits[:,-1,:] #B,C
            probs = F.softmax(logits,dim=1) #B,C
            idx_next = torch.multinomial(probs,num_samples=1) #B,1
            idx = torch.cat((idx,idx_next),dim=1)
        return idx
    
m = BigramLLM(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)
        
        
        
        

torch.Size([32, 51])
tensor(4.2270, grad_fn=<NllLossBackward0>)


In [279]:
x = torch.tensor([encode("hi murph")])
#x.unsqueeze(dim=1)
#x.unsqueeze(dim=0)
x.shape

torch.Size([1, 8])

In [280]:
decode(m.generate(x,max_new_tokens=100)[0].tolist())

'hi murphvyzf0sh54!lg05 l\nï¿\n1f»72-»r .c55!hz3oztcn1q9!lew4gvykeegy8q»hkujwj9rkzloyosufm8hbi,yhkp2r\n7o1w1ãscl'

In [283]:
optimizer = torch.optim.Adam(m.parameters(),lr=1e-3)

In [285]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    out,loss = m(xb,yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(loss.item())
    


2.3369948863983154


In [287]:
decode(m.generate(x,max_new_tokens=500)[0].tolist())

"hi murphed be, co on y ioule al wefoous\no f cas e palidayoredos.\noks bowone.\nis hink. anerkilit?\nyolistod ioornitahthen blomery anse ak he. atopepli ph. h.\nwhominack.\nisouso pe d.\n- il adnonecoup in..\nil whet cthandollis?\n- theabethe th arkn.\nare\ny, ld wh, tory, moon'vok.\nht\n- me w whee d whe dilio het.\nthep.\na.\nwe isit anil yow s inemhee drore gexrtin op, onthet sstet beticurakncanest'meathehasth wing dath.\nhalyo deeent illng,\nysalore ce 4»fow!\nrealyof iom- ong we thyeandom isedng byou's w,\nday in, mea"

In [288]:
###attention

In [302]:
b = torch.randint(0,10,(3,1)).float()
a = torch.tril(torch.ones(3,3))
a = a/a.sum(dim=1,keepdim=True)
c = a @ b
print(c)
print(b)


tensor([[6.0000],
        [7.0000],
        [7.6667]])
tensor([[6.],
        [8.],
        [9.]])


In [309]:
##attention @ B,T,C = B,T,C
##ATTENTION = [t,t]
B,T,C = 4,8,2
x = torch.randn((4,8,2))
att = torch.tril(torch.ones(T,T))
att = att/att.sum(dim=1)
out = att @ x
out.shape
print(att)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.5000, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.5000, 0.3333, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.5000, 0.3333, 0.2500, 0.2000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.5000, 0.3333, 0.2500, 0.2000, 0.1667, 0.0000, 0.0000],
        [1.0000, 0.5000, 0.3333, 0.2500, 0.2000, 0.1667, 0.1429, 0.0000],
        [1.0000, 0.5000, 0.3333, 0.2500, 0.2000, 0.1667, 0.1429, 0.1250]])


In [313]:
a = torch.tril(torch.ones(3,3))
a

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [321]:
#Attention sfter embedding on  B T C

B,T,C=4,8,32

x = torch.randn((B,T,C))

head_size = 16

key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)

k = key(x) #b t h
q = query(x) #b t h

wei =  q @ k.transpose(-2,-1) # b t 16 b 16 t = b t t

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei2 = wei.masked_fill(tril[:T][:T]==0,float('-inf'))

wei = F.softmax(wei,dim=-1)
wei2 = F.softmax(wei2,dim=-1)
v = value(x) # b t h

out1 = wei @ v # b t h 
out2 = wei2 @ v

(out2==out1).all()

tensor(True)

In [315]:
class LayerNorm1d(nn.Module):
    def __init__(self,dim,eps=1e-5,momentum=0.1):
        super().__init__()
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    def forward(self,x):
        xmean = x.mean(1,keepdim=True)
        xvar = x.var(1,keepdim=True)
        xhat = (x-xmean)/torch.sqrt(xvar+self.eps)
        self.out = self.gamma*xhat + self.beta
        return self.out
model = LayerNorm1d(100)
x = torch.randn((32,100))
model(x).shape
        

torch.Size([32, 100])

In [319]:
x = torch.randn((12,100))
a  = x.mean(1)
a[0] == x[0,:].mean()


tensor(True)

step 100: train loss nan, val loss nan
step 200: train loss nan, val loss nan
step 300: train loss nan, val loss nan
step 400: train loss nan, val loss nan
step 500: train loss nan, val loss nan
step 600: train loss nan, val loss nan
step 700: train loss nan, val loss nan
step 800: train loss nan, val loss nan
step 900: train loss nan, val loss nan
step 1000: train loss nan, val loss nan
step 1100: train loss nan, val loss nan
step 1200: train loss nan, val loss nan
step 1300: train loss nan, val loss nan
step 1400: train loss nan, val loss nan
step 1500: train loss nan, val loss nan
step 1600: train loss nan, val loss nan
step 1700: train loss nan, val loss nan
step 1800: train loss nan, val loss nan
step 1900: train loss nan, val loss nan
step 2000: train loss nan, val loss nan
step 2100: train loss nan, val loss nan
step 2200: train loss nan, val loss nan
step 2300: train loss nan, val loss nan
step 2400: train loss nan, val loss nan
step 2500: train loss nan, val loss nan
step 2600

In [430]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64*2
n_head = 4*2
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('interstellar.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
           # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):#

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.814151 M parameters
step 0: train loss 4.5189, val loss 4.5159
step 100: train loss 2.5067, val loss 2.5601
step 200: train loss 2.3618, val loss 2.4195
step 300: train loss 2.2149, val loss 2.2706
step 400: train loss 2.0944, val loss 2.1350
step 500: train loss 1.9943, val loss 2.0170
step 600: train loss 1.8891, val loss 1.9501
step 700: train loss 1.8330, val loss 1.9113
step 800: train loss 1.7832, val loss 1.8471
step 900: train loss 1.7246, val loss 1.8286
step 1000: train loss 1.6757, val loss 1.8153
step 1100: train loss 1.6364, val loss 1.7592
step 1200: train loss 1.6001, val loss 1.7766
step 1300: train loss 1.5720, val loss 1.7534
step 1400: train loss 1.5207, val loss 1.7216
step 1500: train loss 1.5088, val loss 1.7345
step 1600: train loss 1.4633, val loss 1.7140
step 1700: train loss 1.4481, val loss 1.7001
step 1800: train loss 1.4248, val loss 1.7046
step 1900: train loss 1.3963, val loss 1.7187
step 2000: train loss 1.3776, val loss 1.6920
step 2100: train loss 1.

In [431]:
from pathlib import Path

# 1. Create models directory 
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path 
MODEL_NAME = "01_pytorch_Interstellar_gpt.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict 
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH) 

Saving model to: models\01_pytorch_Interstellar_gpt.pth


In [440]:
prompt = "do not go gentle into that good night  "
x = torch.tensor([encode(prompt)])
#x.unsqueeze(dim=1)
#x.unsqueeze(dim=0)
#x.shape
print(decode(m.generate(x,max_new_tokens=500)[0].tolist()))

do not go gentle into that good night  Maype.
But who put to you in a messade very.
So, it's bad?
No, I need for hamp.
He a sure could use hibeuy a persock to this is?
Deate. Be them.
He seement to see at the food.
The only thing in hids fuel to visit back forrom the corlice. Any just a fine.
All thing in besefore you a makin a back for.
When I was this?
Dad?
It's golad Molles
you difter my family,
Kuddat something in back.
He leave it me.
If you leave you left legwn here, Grand?
"We aballed's peable. I was all a Day!
The nights afte
